In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
complaints_df = pd.read_excel("/content/drive/MyDrive/dataset/Copy of Team 3 - Complaints.xlsx")

# Preprocess the narratives
def preprocess_narrative(narrative):
    if pd.isna(narrative):
        return ''
    # Remove non-alphabetic characters and convert to lowercase
    clean_narrative = re.sub(r'[^a-zA-Z\s]', '', str(narrative).lower())
    return clean_narrative

complaints_df['processed_narrative'] = complaints_df['narrative'].apply(preprocess_narrative)

# Tokenize narratives
tokenized_narratives = complaints_df['processed_narrative'].apply(lambda x: x.split())

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_narratives, vector_size=100, window=5, min_count=1, workers=4)

# Generate word embeddings
def generate_word_embeddings(narrative, model):
    word_embeddings = []
    for word in narrative.split():
        if word in model.wv:
            word_embeddings.append(model.wv[word])
    return np.mean(word_embeddings, axis=0) if word_embeddings else np.zeros(model.vector_size)

complaints_df['word_embeddings'] = complaints_df['processed_narrative'].apply(lambda x: generate_word_embeddings(x, word2vec_model))

# Split data into features (Word2Vec embeddings) and target labels
X = np.vstack(complaints_df['word_embeddings'])
y = complaints_df['product']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
dtree = DecisionTreeClassifier(max_depth=20)
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)


Accuracy: 0.7879
Classification Report:
                     precision    recall  f1-score   support

        credit_card       0.58      0.57      0.57      3212
   credit_reporting       0.88      0.91      0.89     18130
    debt_collection       0.68      0.62      0.65      4619
mortgages_and_loans       0.72      0.69      0.70      3738
     retail_banking       0.68      0.67      0.67      2786

           accuracy                           0.79     32485
          macro avg       0.71      0.69      0.70     32485
       weighted avg       0.78      0.79      0.79     32485

